[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=ShowPerpEventFillsPolling.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=ShowPerpEventFillsPolling.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Perp Event Fill Polling

This notebook shows how to fetch poll the event queue for a perp market, and list all the new fills for a given account.

You may be best using the same account in another window to trade so that there are some fills to show.

In [ ]:
import entropy
import time

# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Create a 'mainnet' Context
with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    # Entropy accounts are per-Group, so we need to load the Group first.
    group = entropy.Group.load(context)

    # Get all the Wallet's accounts for that Group
    accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)

    # Just pick the first account even if there are multiple accounts
    account = accounts[0]

    # Load the market
    market = entropy.market(context, "BTC-PERP")

    # Pick out the event queue address
    event_queue_address = market.event_queue_address

    # Load and parse the initial event queue
    initial = entropy.PerpEventQueue.load(context, event_queue_address, market.lot_size_converter)

    # Create the tracker and initialise it with the current state of the event queue.
    tracker = entropy.UnseenAccountFillEventTracker(initial, account.address)

    # Now loop - sleep then poll for new fills
    for counter in range(5):
        time.sleep(5)
        updated = entropy.PerpEventQueue.load(context, event_queue_address, market.lot_size_converter)
        unseen = tracker.unseen(updated)
        print("New fills:", unseen)

print("Example complete.")